# Proyecto Final 

## Introducción 

### DataSet

El dataset utilizado para este proyecto corresponde a 5 archivos que contienen información de las principales ligas europeas y sus jugadores. Esta información corresponde a las temporadas 2008/2009 y 2009/2010 e incluye también detalles acerca de los distintos encuetros que se llevaron a cabo dentro de dicho período, en qué ligas, cuáles fueron los equipos que perticiparon, quiénes formaban parte de la plantilla en ese momento, entre otros. 

Algo que también incluye este dataset y resulta ser informaicón valiosa para fines de análisis de la información, son estadísticas que responden a un desempeño muy específico y de interés para los clubes y entrenadores, al momento de identificar el rendimieno que está teniendo su plantilla a lo largo de un torneo regular. En este sentido, es posible para la persona que esté analizando la información inferir sobre indicadores tales como velocidad, aceleración, capacidad de regate, balance o potencia de tiros. Además también se incluyen calificaciones cualitativas de las capacidades del jugar ya sea en defensa o ataque, así como indicadores tan básicos y pero sumamente importantes como la pierna predilecta del jugador. 

### Principales Tablas 

Las tablas que conforman este conjunto de datos son: 

- Country
- League
- Match
- Player
- Playes_attributes
- Sqlite Master
- Sqlite Sequence
- Team
- Team Attributes 

Dentro de la tabla League se definen los identificadores principales de las ligas más importantes de Europa

![image](https://user-images.githubusercontent.com/99599847/162853492-c5bf1397-ecb9-4da2-9107-3bca3ec214e4.png)

En las tablas de Team y Team Attributes se puede encontrar el consolidado de los equipos con su respectivo id único y id fifa. Los atributos más relevantes para fines de análisis que sobresaltan son los que corresponden al estilo de juego que tiene cada uno de los equipos, características defensivas, de ataque, dribleo, etc. 

![image](https://user-images.githubusercontent.com/99599847/162853676-080e3866-6a5e-4dce-af60-ef906e6fde03.png)

Las tablas de Player y Player_attributes son muy similares a las tablas de equipos, una plantea la información general de los jugadores como nombre, fecha de nacimiento y algunos ids específicos, y otra contiene detalles más específicos que corresponden al performance de los jugadores y skills específicos que definen cuáles son sus atributos principales para el funcionamiento dentro del equipo. 

![image](https://user-images.githubusercontent.com/99599847/162853995-6237f79d-2b2c-4fc3-9a07-4f73bf483862.png)

Por último, la tabla Match consolidado un alto número de identificadores que permite tener el consolidado de todos los juegos y las estadísticas de cada uno de ellos durante las dos temporadas que consolidada el dataset. Acá se muestra información relevante estadísticamente del juego, tan básico como los goles, tiros y corners, hasta indicadores más específicos que  se utilizan a niveles más profundos de análisis del juego y que permiten tener un mejor entendimiento del performance que tuvieron ambos equipos y como pudo ser su evolución a lo largo de ambas temporadas. 

![image](https://user-images.githubusercontent.com/99599847/162854958-932e027f-525a-4c16-a474-a3598494b461.png)

%load_ext sql 

In [ ]:
DB_ENDPOINT = 'proyecto-cdp-v3.cacnlwtsjcuo.us-east-1.rds.amazonaws.com'
DB = 'schema_proyectocdp'
DB_USER = 'admin'
DB_PASSWORD = 'proyectocdp123'
DB_PORT = '3306'

In [ ]:
mysql_conn = 'mysql+pymysql://{}:{}@{}/{}'.format(DB_USER, DB_PASSWORD, DB_ENDPOINT, DB)
print(mysql_conn)

In [ ]:
%sql $mysql_conn #Solicitud de conexión 

In [ ]:
%%sql
SELECT * FROM main_Country

## Usando Python

In [ ]:
import pandas as pd

sql_query = 'SELECT * FROM main_Country;'
dfCountry = pd.read_sql(sql_query, mysql_conn)

sql_query = 'SELECT * FROM main_League;'
dfLeague = pd.read_sql(sql_query, mysql_conn)

sql_query = 'SELECT * FROM main_Match;'
dfMatch = pd.read_sql(sql_query, mysql_conn)

sql_query = 'SELECT * FROM main_Player;'
dfPlayer = pd.read_sql(sql_query, mysql_conn)

sql_query = 'SELECT * FROM main_Player_Attributes;'
dfPlayerAttribute = pd.read_sql(sql_query, mysql_conn)

sql_query = 'SELECT * FROM main_Team;'
dfTeam = pd.read_sql(sql_query, mysql_conn)

sql_query = 'SELECT * FROM main_Team_Attributes;'
dfTeamAttribute = pd.read_sql(sql_query, mysql_conn)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dfCountry_League = dfCountry.merge(dfLeague, left_on='id', right_on='country_id')
dfCountry_League.head()

### Construcción dim_League

In [ ]:
dimLeague = dfCountry_League.loc[: , ['id', 'country_name','league_id','league_name']]
dimLeague.head()

In [ ]:
len(dimLeague)

### Construcción dim_Team

In [ ]:
len(dimTeam)

In [ ]:
dfTeam_TeamAttri = dfTeam.merge(dfTeamAttribute, left_on='team_api_id', right_on='team_api_id')
dfTeam_TeamAttri.head()

In [ ]:
dimTeam = dfTeam_TeamAttri.loc[: , ['id',
'team_api_id',
'team_fifa_api_id_x',
'team_log_name',
'team_short:_name',
]]
dimTeam.head()

In [ ]:
sql_query = 'SELECT * FROM main_Team;'
dfTeam = pd.read_sql(sql_query, mysql_conn)
dfTeam.head()

## dimDate con Python

In [ ]:
sql_query = 'SELECT date FROM main_Match;'
dfDate = pd.read_sql(sql_query, mysql_conn)
dfDate['date'] = dfDate['date'].dt.date 
dfDate.head()

In [ ]:
dfDate['year'] = pd.DatetimeIndex(dfDate['date']).year
dfDate['month'] = pd.DatetimeIndex(dfDate['date']).month
dfDate['quarter'] = pd.DatetimeIndex(dfDate['date']).quarter
dfDate['day'] = pd.DatetimeIndex(dfDate['date']).day
dfDate['week'] = pd.DatetimeIndex(dfDate['date']).week
dfDate['is_weekend'] = dfDate['dayofweek'].apply(lambda x: 1 if x > 5 else 0)
dfDate['dayofweek'] = pd.DatetimeIndex(dfDate['date']).dayofweek
dfDate['date'] = pd.to_datetime(dfDate.date, format='%Y-%m-%d')
dfDate['date_key'] = dfDate['date'].dt.strftime('%Y%M%d') #Se define date_key para ser la llave foranea en la fact table
dfDate.head()

In [ ]:
dimDate = dfDate.loc[:, ['date_key'
                         , 'date'
                         , 'year'
                         , 'month'
                         , 'day'
                         , 'is_weekend']]
dimDate.head()

 ### Creación Dim Player

In [ ]:
dfPlayer_PlayerAttri = dfPlayer.merge(dfPlayerAttribute, left_on='player_api_id', right_on='player_api_id')
dfPlayer_PlayerAttri.head()

In [ ]:
dimPlayer = dfPlayer_PlayerAttri.loc[: , ['id',
'player_api_id',
'player_name',
'player_fifa_api_id_x',
'birthday',
'height',
'weight',
]]
dimTeam.head()

### Creación Fact Table


In [ ]:
dfLeague_Match = dfMatch.merge(dimLeague, left_on='league_id', right_on='id')
dfLeague_Match.head()

In [ ]:
len(dfLeague_Match)

In [ ]:
dfMatch.head()

In [ ]:
dfLeague_Match.info()

In [ ]:
dfTeam_Match = dfLeague_Match.merge(dimTeam, left_on=['home_team_api_id'], right_on=['team_api_id'])

In [ ]:
dfTeam_Match.info()

In [ ]:
dfTeam_Match_a = dfTeam_Match.merge(dimTeam, left_on=['away_team_api_id'], right_on=['team_api_id'])
dfTeam_Match_a.head()

In [ ]:
len(dfTeam_Match_a)

In [ ]:
dfPlayer_Match_b = dimPlayer.merge(dfTeam_Match_a, left_on=['id'] , right_on=['away_player_X1'
])
dfPlayer_Match_c = dimPlayer.merge(dfPlayer_Match_b, left_on=['id'] , right_on=['away_player_X2'
])
dfPlayer_Match_d = dimPlayer.merge(dfPlayer_Match_c, left_on=['id'] , right_on=['away_player_X3'
])
dfPlayer_Match_e = dimPlayer.merge(dfPlayer_Match_d, left_on=['id'] , right_on=['away_player_X4'
])
dfPlayer_Match_f = dimPlayer.merge(dfPlayer_Match_e, left_on=['id'] , right_on=['away_player_X5'
])
dfPlayer_Match_g = dimPlayer.merge(dfPlayer_Match_f, left_on=['id'] , right_on=['away_player_X6'
])
dfPlayer_Match_h = dimPlayer.merge(dfPlayer_Match_g, left_on=['id'] , right_on=['away_player_X7'
])
dfPlayer_Match_i = dimPlayer.merge(dfPlayer_Match_h, left_on=['id'] , right_on=['away_player_X8'
])
dfPlayer_Match_j = dimPlayer.merge(dfPlayer_Match_i, left_on=['id'] , right_on=['away_player_X9'
])
dfPlayer_Match_k = dimPlayer.merge(dfPlayer_Match_j, left_on=['id'] , right_on=['away_player_X10'
])
dfPlayer_Match_l = dimPlayer.merge(dfPlayer_Match_k, left_on=['id'] , right_on=['away_player_X11'
])

In [ ]:
len(dfPlayer_Match_l)

In [ ]:
FactMatch = dfPlayer_Match_l.loc[: , ['id',
'player_api_id',
'player_fifa_api_id_x',                                
'team_api_id_x',
'league_id_x', 
'country_id',
'season',
'stage',
'date',
'home_team_api_id',
'away_team_api_id',
'home_team_goal',
'away_team_goal',
'home_player_X1',
'home_player_X2',
'home_player_X3',
'home_player_X4',
'home_player_X5',
'home_player_X6',
'home_player_X7',
'home_player_X8',
'home_player_X9',
'home_player_X10',
'home_player_X11',
'away_player_X1',
'away_player_X2',
'away_player_X3',
'away_player_X4',
'away_player_X5',
'away_player_X6',
'away_player_X7',
'away_player_X8',
'away_player_X9',
'away_player_X10',
'away_player_X11',
'home_player_Y1',
'home_player_Y2',
'home_player_Y3',
'home_player_Y4',
'home_player_Y5',
'home_player_Y6',
'home_player_Y7',
'home_player_Y8',
'home_player_Y9',
'home_player_Y10',
'home_player_Y11',
'away_player_Y1',
'away_player_Y2',
'away_player_Y3',
'away_player_Y4',
'away_player_Y5',
'away_player_Y6',
'away_player_Y7',
'away_player_Y8',
'away_player_Y9',
'away_player_Y10',
'away_player_Y11',
'home_player_1',
'home_player_2',
'home_player_3',
'home_player_4',
'home_player_5',
'home_player_6',
'home_player_7',
'home_player_8',
'home_player_9',
'home_player_10',
'home_player_11',
'away_player_1',
'away_player_2',
'away_player_3',
'away_player_4',
'away_player_5',
'away_player_6',
'away_player_7',
'away_player_8',
'away_player_9',
'away_player_10',
'away_player_11',
'goal',
'shoton',
'shotoff',
'foulcommit',
'card',
'cross',
'corner',
'possession',
'B365H',
'B365D',
'B365A',
'BWH',
'BWD',
'BWA',
'IWH',
'IWD',
'IWA',
'LBH',
'LBD',
'LBA',
'PSH',
'PSD',
'PSA',
'WHH',
'WHD',
'WHA',
'SJH',
'SJD',
'SJA',
'VCH',
'VCD',
'VCA',
'GBH',
'GBD',
'GBA',
'BSH',
'BSD',
'BSA',                                  
]]

FactMatch.head()

In [ ]:
FactMatch.info()

### Importar a Redshift

In [ ]:
import os
import configparser